# 3 этап создание модели 

* В данном ноутбуке мы сделаем следующее:

    - Обработаем и отнормируем признаки
    - Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
    - Построим модель при помощи LinearRegression
    - Обучим модель на основе случайного леса RandomForestRegressor
    - Обучим модель с L1 и L2 регуляризаций ElasticNetCV
    - Сделаем модель градиентного бустинга CatBoostRegressor
    - На основе предыдущих шагов выберем оптимальную модель

In [1]:
import random
import numpy as np 
import pandas as pd 
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor


import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

C:\Users\Koop_\AppData\Local\Temp\ipykernel_22484\101661651.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [3]:
data = pd.read_csv('data/data_model.csv')
display(data.head())
data.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,Active,2.0,Mason City,3588,50401,IA,244900,False,single_family_home,1970,True,True,False,False,3.8,5.60
3,Other,3.0,Houston,1930,77080,TX,311995,False,single_family_home,2019,True,True,True,False,3.0,0.60
4,For Sale,2.0,Flushing,1300,11354,NY,669000,False,condo,1965,False,False,True,False,2.8,0.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210819 entries, 0 to 210818
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   status              210819 non-null  object 
 1   baths               210819 non-null  float64
 2   city                210819 non-null  object 
 3   sqft                210819 non-null  int64  
 4   zipcode             210819 non-null  int64  
 5   state               210819 non-null  object 
 6   target              210819 non-null  int64  
 7   pool_encoded        210819 non-null  bool   
 8   Type                210819 non-null  object 
 9   Year built          210819 non-null  object 
 10  Heating_encoded     210819 non-null  bool   
 11  Cooling_encoded     210819 non-null  bool   
 12  Parking_encoded     210819 non-null  bool   
 13  fireplace_encoded   210819 non-null  bool   
 14  school_rating_mean  210819 non-null  float64
 15  school_dist_min     210819 non-nul

In [4]:
# Составим список булевых признаков:
bin_features = ['pool_encoded','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','zipcode','state','Type','Year built']
 
# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating_mean', 'school_dist_min']

In [5]:
# подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = data[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1531
Количество уникальных значений в категориальной колонке zipcode: 3962
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [6]:
def preproc_data(df_input):
    '''Функция для предварительной обработки данных предикторов.'''
    # Создаем копию входного DataFrame, чтобы избежать изменений в оригинале
    df_output = df_input.copy()
    
    # Преобразуем столбец зип-кодов в строковый тип, чтобы он стал категориальным
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    
    # Преобразуем столбец года постройки в строковый тип, чтобы он стал категориальным
    df_output['Year built'] = df_output['Year built'].astype(str)
    
    # Нормализация и логарифмирование числовых данных
    for column in ['baths', 'sqft', 'target', 'school_rating_mean', 'school_dist_min']:
        # Применяем модуль к значениям, чтобы избежать негативных значений перед логарифмированием
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        # Добавляем малую константу, чтобы избежать логарифмирования нуля
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
    
    # Кодирование категориальных признаков с помощью One-Hot Encoding
    ohe_status = OneHotEncoder(sparse_output=False)
    ohe_state = OneHotEncoder(sparse_output=False)
    ohe_Type = OneHotEncoder(sparse_output=False)
    
    # Применяем One-Hot Encoding к столбцам 'status', 'state' и 'Type'
    status_ohe = ohe_status.fit_transform(df_output['status'].values.reshape(-1, 1))
    state_ohe = ohe_state.fit_transform(df_output['state'].values.reshape(-1, 1))
    Type_ohe = ohe_Type.fit_transform(df_output['Type'].values.reshape(-1, 1))
    
    # Кодирование категориальных признаков с помощью Label Encoding
    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])
    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])
    city_le = LabelEncoder()
    city_label = city_le.fit_transform(df_output['city'])
    zip_le = LabelEncoder()
    zip_label = zip_le.fit_transform(df_output['zipcode'])
    
    # Добавляем закодированные категориальные признаки в DataFrame
    df_output = df_output.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
    
    # Добавляем закодированные метки в DataFrame
    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_label'] = city_label
    df_output['zip_label'] = zip_label
    
    # Удаляем оригинальные категориальные столбцы
    df_output.drop(['status', 'state', 'Type', 'city', 'zipcode', 'Year built'], axis=1, inplace=True)
    
    return df_output

In [7]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(data)
df_encoded.sample(10)

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min,...,Type_modern,Type_multi_family_home,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_label,zip_label
109016,6.931477e-01,7.178545,12.432810,False,False,False,False,False,0.693148,2.191654,...,0.0,0.0,0.0,0.0,1.0,0.0,1,200,479,3517
108554,6.931477e-01,7.329094,12.555535,False,False,True,True,False,1.667707,-1.203969,...,0.0,0.0,0.0,0.0,1.0,0.0,5,156,1338,1141
43136,1.386295e+00,7.491088,12.754165,False,True,True,True,False,0.693148,0.190621,...,0.0,0.0,0.0,0.0,1.0,0.0,28,201,614,2488
140334,6.931477e-01,7.314553,12.230765,False,True,True,True,True,2.079442,0.076962,...,0.0,0.0,0.0,0.0,1.0,0.0,22,175,1323,1610
60726,9.999995e-07,7.073270,11.453048,False,True,True,True,False,1.098613,1.308333,...,0.0,0.0,0.0,0.0,1.0,0.0,21,120,182,242
209301,6.931477e-01,7.272398,12.097925,False,True,True,True,False,0.693148,0.832910,...,0.0,0.0,0.0,0.0,1.0,0.0,28,201,1190,2638
31109,1.098613e+00,7.748891,13.091904,False,True,True,True,True,-0.693145,-0.713348,...,0.0,0.0,0.0,0.0,1.0,0.0,1,190,479,3525
131847,1.098613e+00,7.625595,12.936034,False,True,True,False,True,0.993252,-1.714793,...,0.0,0.0,0.0,0.0,1.0,0.0,5,108,1357,1158
147621,6.931477e-01,7.167809,12.574182,False,True,True,True,False,1.609438,-0.967581,...,0.0,0.0,0.0,0.0,1.0,0.0,20,184,1110,3122
118231,6.931477e-01,7.004882,12.429216,False,True,True,False,False,1.335001,-2.040213,...,0.0,0.0,0.0,0.0,0.0,0.0,5,168,582,1023


In [8]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210819 entries, 0 to 210818
Data columns (total 72 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    210819 non-null  float64
 1   sqft                     210819 non-null  float64
 2   target                   210819 non-null  float64
 3   pool_encoded             210819 non-null  bool   
 4   Heating_encoded          210819 non-null  bool   
 5   Cooling_encoded          210819 non-null  bool   
 6   Parking_encoded          210819 non-null  bool   
 7   fireplace_encoded        210819 non-null  bool   
 8   school_rating_mean       210819 non-null  float64
 9   school_dist_min          210819 non-null  float64
 10  status_Active            210819 non-null  float64
 11  status_Auction           210819 non-null  float64
 12  status_Back on Market    210819 non-null  float64
 13  status_Coming Soon       210819 non-null  float64
 14  stat

In [9]:
# Разделим датасет
y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Обучение модели
## Model 1: Создадим "наивную" модель

Эта модель будет предсказывать среднюю стоимость по общей площади и городу. C ней будем сравнивать другие модели.

In [10]:
class NaiveModel:
    def __init__(self):
        self.means = None
    def fit(self, X, y):
        # Проверка на наличие необходимых данных
        if X is None or y is None:
            raise ValueError("Входные данные X и y не должны быть пустыми.")
        
        # Создаем DataFrame из входных данных
        X_df = pd.DataFrame(X, columns=['city_label'])
        y_df = pd.DataFrame(y, columns=['target'])
        
        # Объединяем данные и вычисляем средние значения по городам
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby('city_label')['target'].mean().reset_index()
    def predict(self, X):
        # Проверка, обучена ли модель
        if self.means is None:
            raise ValueError("Модель не обучена. Пожалуйста, вызовите метод fit().")
        
        # Создаем копию входных данных
        X = pd.DataFrame(X, columns=['city_label']).copy()
        X['mean'] = np.nan
        
        # Присваиваем средние значения
        mean_map = dict(zip(self.means['city_label'], self.means['target']))
        X['mean'] = X['city_label'].map(mean_map)
        
        # Заполняем пропуски средним значением по всему набору данных
        overall_mean = X['mean'].mean()
        X['mean'].fillna(overall_mean, inplace=True)
        
        return X['mean'].to_numpy()
# Инициализация и обучение модели
naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
# Предсказания и оценка модели
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)
# Вычисление метрик
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
# Вывод метрик
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.43
Test MSE: 0.42
Train MAE: 0.48
Test MAE: 0.48
Train R2: -0.01
Test R2: -0.01


- Среднеквадратичная ошибка (MSE):


    Train MSE: 0.43
    Test MSE: 0.42

    MSE показывает среднюю квадратичную разницу между предсказанными и фактическими значениями. Значения MSE для обучающей и тестовой выборок близки друг к другу, что указывает на отсутствие переобучения. Однако, сами по себе эти значения не дают полной картины без контекста диапазона целевой переменной.


- Средняя абсолютная ошибка (MAE):


    Train MAE: 0.48
    Test MAE: 0.48

    MAE показывает среднюю абсолютную разницу между предсказанными и фактическими значениями. Поскольку значения MAE для обучающей и тестовой выборок также равны, это указывает на стабильность модели. Однако, как и в случае с MSE, без контекста сложно сказать, насколько это хорошее значение.


- Коэффициент детерминации (R²):


    Train R2: -0.01
    Test R2: -0.01

    R² показывает, какую долю дисперсии целевой переменной объясняет модель. Значение R² около 0 указывает на то, что модель практически не объясняет вариацию целевой переменной. Значение -0.01 говорит о том, что модель хуже, чем простая модель, которая всегда предсказывает среднее значение целевой переменной.

Наивная модель не является эффективной для данной задачи, так как она практически не объясняет вариацию данных (R² около 0).


## Model 2: LinearRegression

In [11]:
# Создаем и обучаем модель линейной регрессии
model = LinearRegression(fit_intercept=False)
try:
    model.fit(X_train, y_train)
except Exception as e:
    print(f"Ошибка при обучении модели: {e}")
    raise
# Делаем предсказания
try:
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
except Exception as e:
    print(f"Ошибка при предсказании: {e}")
    raise
# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.23
Test MSE: 0.23
Train MAE: 0.35
Test MAE: 0.34
Train R2: 0.45
Test R2: 0.45


- Среднеквадратичная ошибка (MSE):


    Train MSE: 0.23
    Test MSE: 0.23

    Значения MSE для обучающей и тестовой выборок равны и ниже, чем у наивной модели (0.43 и 0.42 соответственно). Это указывает на то, что линейная регрессия лучше справляется с задачей предсказания, чем наивная модель.


- Средняя абсолютная ошибка (MAE):


    Train MAE: 0.35
    Test MAE: 0.34

    Значения MAE также ниже, чем у наивной модели (0.48 для обеих выборок), что говорит о более точных предсказаниях линейной регрессии.


- Коэффициент детерминации (R²):


    Train R2: 0.45
    Test R2: 0.45

    Значение R² равно 0.45 для обеих выборок, что значительно лучше, чем у наивной модели (-0.01). Это указывает на то, что линейная регрессия объясняет 45% вариации целевой переменной, что является значительным улучшением.


- Выводы:
    * Улучшение по сравнению с наивной моделью: Линейная регрессия показывает значительно лучшие результаты по всем метрикам по сравнению с наивной моделью.

    * Стабильность: Значения метрик для обучающей и тестовой выборок очень близки, что указывает на отсутствие переобучения и хорошую обобщающую способность модели.
    * Объясненная вариация: Хотя R² не равен 1, значение 0.45 является хорошим показателем, учитывая, что мы работаем с простой линейной моделью. Это означает, что модель объясняет значительную часть вариации в данных.

## Model 3: RandomForestRegressor

In [12]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)
# Задаем диапазон гиперпараметров для настройки
param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
# Настраиваем гиперпараметры с помощью RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=RANDOM_SEED
)
# Обучаем модель на обучающих данных
random_search.fit(X_train, y_train)
# Получаем лучшую модель
best_rf_regressor = random_search.best_estimator_
# Предсказания на обучающих и тестовых данных
y_train_pred = best_rf_regressor.predict(X_train)
y_test_pred = best_rf_regressor.predict(X_test)
# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.02
Test MSE: 0.08
Train MAE: 0.09
Test MAE: 0.18
Train R2: 0.96
Test R2: 0.81


* Результаты модели RandomForestRegressor показывают значительное улучшение по сравнению с предыдущими моделями. Давайте проанализируем полученные метрики:

    - Среднеквадратичная ошибка (MSE):


        Train MSE: 0.02
        Test MSE: 0.08

        Низкие значения MSE как для обучающей, так и для тестовой выборок указывают на высокую точность модели. Разница между обучающей и тестовой MSE относительно небольшая, что говорит о хорошем обобщении модели на новых данных.


    - Средняя абсолютная ошибка (MAE):


        Train MAE: 0.9
        Test MAE: 0.18

        Значения MAE также низкие, что свидетельствует о том, что средняя ошибка предсказания модели мала. Это подтверждает, что модель делает точные предсказания.


    - Коэффициент детерминации (R²):


        Train R2: 0.96
        Test R2: 0.81

        Высокое значение R² для обучающей выборки (0.96) говорит о том, что модель объясняет 96% вариации целевой переменной. Значение R² для тестовой выборки (0.81) также высокое, что указывает на хорошую обобщающую способность модели.


- Выводы:
    * Высокая точность: Модель RandomForestRegressor демонстрирует высокую точность и способность хорошо обобщать на новых данных.
    * Стабильность: Разница между метриками на обучающей и тестовой выборках относительно небольшая, что указывает на отсутствие переобучения.
    * Объяснение вариации: Модель объясняет значительную часть вариации в данных (81% на тестовой выборке), что является отличным показателем для регрессионной задачи.

## Model 4: ElasticNetCV

In [13]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(
    cv=5,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    alphas=np.logspace(-4, 4, 100),
    l1_ratio=np.linspace(0.1, 1.0, 10)
)
model_el.fit(X_train, y_train)
# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)
# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.23
Test MSE: 0.23
Train MAE: 0.35
Test MAE: 0.34
Train R2: 0.45
Test R2: 0.45


- Среднеквадратичная ошибка (MSE):


    Train MSE: 0.23
    Test MSE: 0.23

    Значения MSE для обучающей и тестовой выборок равны и умеренные. Это говорит о том, что модель делает предсказания с постоянной точностью на обоих наборах данных.


- Средняя абсолютная ошибка (MAE):


    Train MAE: 0.35
    Test MAE: 0.34

    Значения MAE также равны для обеих выборок и умеренные. Это указывает на то, что средняя ошибка предсказания модели относительно мала.


- Коэффициент детерминации (R²):


    Train R2: 0.45
    Test R2: 0.45

    Значение R² равно 0.45 для обеих выборок, что указывает на то, что модель объясняет 45% вариации целевой переменной. Это неплохой результат, но есть потенциал для улучшения.


- Выводы:
    * Стабильность: Модель показывает стабильные результаты на обучающей и тестовой выборках, что указывает на отсутствие переобучения.
    * Объяснение вариации: Хотя R² не равен 1, значение 0.45 является удовлетворительным показателем, но указывает на то, что модель может быть улучшена для объяснения большей части вариации в данных.
    * Сравнение с другими моделями: Если сравнивать с результатами модели RandomForestRegressor, которая показала более высокие R² и более низкие ошибки, ElasticNetCV может быть менее эффективной для этой задачи.

## Model 5: CatBoostRegressor

In [14]:
def log_data(df_input):
        
    df_output = df_input.copy()
    # переведем признак зип кода в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация данных и логорифмирование
    #scaler = MinMaxScaler()
    for column in num_features:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
    return df_output

In [15]:
# Запускаем и проверяем, что получилось
df_log = log_data(data)
df_log.sample(5)

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
208444,Active,1.386295,Stowe,7.824046,5672,VT,11.695247,False,condo,2008,True,True,True,False,2.197225,1.252763
132572,For Sale,1.098613,Flushing,7.766417,48433,MI,12.277928,False,single_family_home,1973,True,True,True,False,1.740466,0.875469
192015,Other,0.693148,Gulfport,7.130899,33707,FL,12.333586,True,condo,1971,True,True,True,False,0.993252,-0.105359
144742,For Sale,1.098613,Arlington,7.713785,76013,TX,12.429176,False,single_family_home,1979,True,True,True,True,1.840550,-1.609433
57577,For Sale,1.386295,Chicago,6.396930,60608,IL,13.014778,False,multi_family_home,1885,True,False,True,False,1.252763,-1.771951


In [16]:
y = df_log.target.values
X = df_log.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [21]:
param_grid = {
    'iterations': [25, 75, 125],
    'learning_rate': [0.015, 0.04, 0.1],
    'depth': [3, 4, 5],
    'l2_leaf_reg': [1, 3, 5],
}
cb_model = CatBoostRegressor(random_seed=RANDOM_SEED, silent=True)
grid_search = GridSearchCV(estimator=cb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train, cat_features=cat_features)
best_params = grid_search.best_params_
best_cb_model = CatBoostRegressor(iterations=best_params['iterations'], learning_rate=best_params['learning_rate'], depth=best_params['depth'], l2_leaf_reg=best_params['l2_leaf_reg'], random_seed=RANDOM_SEED, silent=True)
best_cb_model.fit(X_train, y_train, cat_features=cat_features)

y_train_pred = best_cb_model.predict(X_train)
y_test_pred = best_cb_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Train MSE: 0.10
Test MSE: 0.11
Train MAE: 0.23
Test MAE: 0.23
Train R2: 0.76
Test R2: 0.74


- Среднеквадратичная ошибка (MSE):


    Train MSE: 0.10
    Test MSE: 0.11

    Значения MSE, что указывает на хороший уровень показатель  предсказанными и фактическими значениями.


- Средняя абсолютная ошибка (MAE):


    Train MAE: 0.23
    Test MAE: 0.23

    Значения MAE показывают среднюю ошибку предсказания,разницы между тестовой и тренеровочной выборкой нет, это говорит о стабильности модели.


- Коэффициент детерминации (R²):


    Train R2: 0.76
    Test R2: 0.74

    Значения R² показывают, что модель объясняет 76% вариации целевой переменной на обучающей выборке и 74% на тестовой. Это хороший результат, указывающий на способность модели объяснять большую часть вариации в данных.


- Выводы:
    * Стабильность: Модель показывает стабильные результаты на обучающей и тестовой выборках, что указывает на отсутствие значительного переобучения.
    * Объяснение вариации: Высокие значения R² показывают, что модель хорошо объясняет вариацию в данных.
    * Высокие ошибки: Высокие значения MSE и MAE могут указывать на наличие выбросов или значительных отклонений в данных.

In [22]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.43, 0.42, 0.48, 0.48, -0.01, -0.01],
        'LinearRegression': [0.23, 0.23, 0.35, 0.34, 0.45, 0.45],
        'RandomForestRegressor': [0.02, 0.08, 0.09, 0.18, 0.96, 0.81],
        'ElasticNetCV': [0.23, 0.23, 0.35, 0.34, 0.45, 0.45],
        'CatBoostRegressor':[0.10, 0.11, 0.23, 0.23, 0.76, 0.74]}

df_metric = pd.DataFrame(data)
df_metric.head(6)

,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV,CatBoostRegressor
0,Train MSE,0.43,0.23,0.02,0.23,0.10
1,Test MSE,0.42,0.23,0.08,0.23,0.11
2,Train MAE,0.48,0.35,0.09,0.35,0.23
3,Test MAE,0.48,0.34,0.18,0.34,0.23
4,Train R2,-0.01,0.45,0.96,0.45,0.76
5,Test R2,-0.01,0.45,0.81,0.45,0.74


На основе приведенных результатов, RandomForestRegressor кажется наилучшей моделью, так как она имеет наименьшие показатели ошибок MSE и MAE, а также высокий коэффициент детерминации ( R^2 ) на тестовых данных. Однако стоит учитывать возможность переобучения, так как разрыв между показателями на тренировочных и тестовых наборах является существенным.

CatBoostRegressor также демонстрирует хорошие результаты и может считаться наилучшим выбором, если переобучение RandomForestRegressor является серьезной проблемой. Ошибки MSE и MAE для CatBoostRegressor на тестовых данных несколько выше, чем у RandomForestRegressor, но разрыв между показателями на тренировочных и тестовых наборах значительно меньше. Коэффициент ( R^2 ) для тестовых данных всего на 0.07 ниже (0.74 против 0.81).

In [23]:
import pickle

# Сохранение выбранной обученной модели в файл pickle
with open("web/app/models/best_cb_model.pkl", "wb") as f:
    pickle.dump(best_cb_model, f)